In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from pprint import pprint
from dataclasses import dataclass, field
from langchain.tools import tool, ToolRuntime
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.chat_models import init_chat_model
from langgraph.types import Command
from langchain.messages import HumanMessage, ToolMessage
from langchain.agents import AgentState
from langchain.agents.middleware import HumanInTheLoopMiddleware, dynamic_prompt, wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

## LLM Model

In [3]:
model = init_chat_model(model="gpt-5-nano")

## Context

In [4]:
@dataclass
class LoginContext:
    users_db: dict = field(default_factory = lambda: {"user1@mail.com": "123",
                                                      "user2@mail.com": "321"})

## State

In [5]:
class AuthState(AgentState):
    authenticated: bool

## Tools

In [6]:
@tool
def check_inbox() -> str:
    """Check the inbox for recent emails"""
    return """
    Hi Julie,
    I'm going to be in town next week and was wondering if we could grab a coffee?
    - best, Jane (jane@example.com)
    """

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send an response email"""
    return f"Email sent to {to} with subject {subject} and body {body}"

In [ ]:
@tool
def authenticate(runtime: ToolRuntime, email: str, password: str):
    """Allow read inbox and send email tools only if user provides correct email and password"""
    if password_db := runtime.context.users_db.get(email):
        if password_db == password:
            return Command(update={
                "authenticated": True,
                "messages": [ToolMessage("Login Successful", tool_call_id=runtime.tool_call_id)]
            }
            )
    return Command(update={
        "authenticated": False,
        "messages": [ToolMessage("Login Failed", tool_call_id=runtime.tool_call_id)]
    }
    )

## MiddleWare

### Wrap Model Call - Dynamic Tool Call

In [9]:
@wrap_model_call
def dynamic_tool_call(request: ModelRequest, handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:
    """Dynamically call tools based on the runtime context"""

    if  request.state.get("authenticated"):
        tools = [check_inbox, send_email]
    else:
        tools = [authenticate]
        request = request.override(tools=tools)
    return handler(request)



### Wrap Model Call - Dynamic Prompt

In [10]:
authenticated_prompt = "You are a helpful assistant that can check the inbox and send emails."
unauthenticated_prompt = "You are a helpful assistant that can authenticate users."

@dynamic_prompt
def dynamic_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on authentication status"""
    authenticated = request.state.get("authenticated")

    if authenticated:
        return authenticated_prompt
    else:
        return unauthenticated_prompt

### Human In The Loop

In [11]:
hitl_middleware = HumanInTheLoopMiddleware(interrupt_on={
                            "authenticate": False,
                            "check_inbox": False,
                            "send_email": True,
                        }
                    )

## Email Agent

In [20]:
agent = create_agent(
    model=model,
    tools=[authenticate, check_inbox, send_email],
    state_schema=AuthState,
    context_schema=LoginContext,
    checkpointer=InMemorySaver(),
    middleware=[dynamic_tool_call, dynamic_prompt, hitl_middleware]
)

## Testing Agent

First request

In [21]:
import threading

config = {"configurable": {"thread_id": threading.get_ident()}}

response = agent.invoke({
        "messages": [HumanMessage(content="Please check my inbox.")],
    },
    context=LoginContext(),
    config=config
)
pprint(response)
print("\n", '-'*100, "\n")
print(response['messages'][-1].content)

{'messages': [HumanMessage(content='Please check my inbox.', additional_kwargs={}, response_metadata={}, id='0948524d-1515-483b-aee2-b2c0bbad36d7'),
              AIMessage(content='I can help with that. I need to sign you in first. Please provide:\n\n- Email address\n- Password\n\nI’ll sign you in and then show your latest messages (unread first by default). If you’d prefer, you can tell me to show only unread, or a specific folder/label.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 776, 'prompt_tokens': 151, 'total_tokens': 927, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 704, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-DAZ7BwEiONloPUABuUV1N0QUOXqhx', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs'

Login - sending wrong login

In [ ]:
#sending a wrong login
response = agent.invoke({
        "messages": [HumanMessage(content="user3@mail.com : 123")],
    },
    context=LoginContext(),
    config=config
)

pprint(response)
print("\n", '-'*100, "\n")
print(response['messages'][-1].content)

{'authenticated': False,
 'messages': [HumanMessage(content='Please check my inbox.', additional_kwargs={}, response_metadata={}, id='0948524d-1515-483b-aee2-b2c0bbad36d7'),
              AIMessage(content='I can help with that. I need to sign you in first. Please provide:\n\n- Email address\n- Password\n\nI’ll sign you in and then show your latest messages (unread first by default). If you’d prefer, you can tell me to show only unread, or a specific folder/label.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 776, 'prompt_tokens': 151, 'total_tokens': 927, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 704, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-DAZ7BwEiONloPUABuUV1N0QUOXqhx', 'service_tier': 'default', 'finish_re

Login - now sending a correct login

In [ ]:
response = agent.invoke({
        "messages": [HumanMessage(content="user1@mail.com : 123")],
    },
    context=LoginContext(),
    config=config
)

pprint(response)
print("\n", '-'*100, "\n")
print(response['messages'][-1].content)

{'authenticated': True,
 'messages': [HumanMessage(content='Please check my inbox.', additional_kwargs={}, response_metadata={}, id='0948524d-1515-483b-aee2-b2c0bbad36d7'),
              AIMessage(content='I can help with that. I need to sign you in first. Please provide:\n\n- Email address\n- Password\n\nI’ll sign you in and then show your latest messages (unread first by default). If you’d prefer, you can tell me to show only unread, or a specific folder/label.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 776, 'prompt_tokens': 151, 'total_tokens': 927, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 704, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-DAZ7BwEiONloPUABuUV1N0QUOXqhx', 'service_tier': 'default', 'finish_rea

Requesting to reply to the email

In [26]:
response = agent.invoke({
        "messages": [HumanMessage(content="Yes, reply to this message")],
    },
    context=LoginContext(),
    config=config
)

pprint(response)
print("\n", '-'*100, "\n")
print(response['messages'][-1].content)

{'__interrupt__': [Interrupt(value={'action_requests': [{'args': {'body': 'Hi '
                                                                          'Jane,\n'
                                                                          '\n'
                                                                          'That '
                                                                          'sounds '
                                                                          'great! '
                                                                          'I’d '
                                                                          'love '
                                                                          'to '
                                                                          'grab '
                                                                          'coffee '
                                                                          'next '
                

Human in the loop interrupt to approve/reject/edit email reply

In [28]:
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

Hi Jane,

That sounds great! I’d love to grab coffee next week. What day and time work best for you?

Looking forward to it,
Julie


Approving the reply

In [ ]:
response = agent.invoke(
    Command(
        resume={"decisions": [{"type": "approve"}]}  # or "reject"
    ),
    config=config
)

pprint(response)
print("\n", '-'*100, "\n")
print(response['messages'][-1].content)

{'authenticated': True,
 'messages': [HumanMessage(content='Please check my inbox.', additional_kwargs={}, response_metadata={}, id='0948524d-1515-483b-aee2-b2c0bbad36d7'),
              AIMessage(content='I can help with that. I need to sign you in first. Please provide:\n\n- Email address\n- Password\n\nI’ll sign you in and then show your latest messages (unread first by default). If you’d prefer, you can tell me to show only unread, or a specific folder/label.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 776, 'prompt_tokens': 151, 'total_tokens': 927, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 704, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-DAZ7BwEiONloPUABuUV1N0QUOXqhx', 'service_tier': 'default', 'finish_rea

: 